# Challenge: Build your own NLP model
 Estimated Time: 4-5 hours



For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

1. Data cleaning / processing / language parsing
2. Create features using two different NLP methods: For example, BoW vs tf-idf.
3. Use the features to fit supervised learning models for each feature set to predict the category outcomes.
4. Assess your models using cross-validation and determine whether one model performed better.
5. Pick one of the models and try to increase accuracy by at least 5 percentage points.

Write up your report in a Jupyter notebook. Be sure to explicitly justify the choices you make throughout, and submit it below.





In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import inaugural, stopwords
from collections import Counter

In [2]:
import nltk
nltk.download('inaugural')

[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Unzipping corpora/inaugural.zip.


True

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(inaugural.fileids())

['1789-Washington.txt', '1793-Washington.txt', '1797-Adams.txt', '1801-Jefferson.txt', '1805-Jefferson.txt', '1809-Madison.txt', '1813-Madison.txt', '1817-Monroe.txt', '1821-Monroe.txt', '1825-Adams.txt', '1829-Jackson.txt', '1833-Jackson.txt', '1837-VanBuren.txt', '1841-Harrison.txt', '1845-Polk.txt', '1849-Taylor.txt', '1853-Pierce.txt', '1857-Buchanan.txt', '1861-Lincoln.txt', '1865-Lincoln.txt', '1869-Grant.txt', '1873-Grant.txt', '1877-Hayes.txt', '1881-Garfield.txt', '1885-Cleveland.txt', '1889-Harrison.txt', '1893-Cleveland.txt', '1897-McKinley.txt', '1901-McKinley.txt', '1905-Roosevelt.txt', '1909-Taft.txt', '1913-Wilson.txt', '1917-Wilson.txt', '1921-Harding.txt', '1925-Coolidge.txt', '1929-Hoover.txt', '1933-Roosevelt.txt', '1937-Roosevelt.txt', '1941-Roosevelt.txt', '1945-Roosevelt.txt', '1949-Truman.txt', '1953-Eisenhower.txt', '1957-Eisenhower.txt', '1961-Kennedy.txt', '1965-Johnson.txt', '1969-Nixon.txt', '1973-Nixon.txt', '1977-Carter.txt', '1981-Reagan.txt', '1985-Reaga

In [0]:

# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [0]:
washington = inaugural.raw('1793-Washington.txt')
kennedy = inaugural.raw('1961-Kennedy.txt')
bush = inaugural.raw('1989-Bush.txt')
clinton = inaugural.raw('1997-Clinton.txt')
obama = inaugural.raw('2009-Obama.txt')

In [0]:
washington = text_cleaner(washington)
kennedy = text_cleaner(kennedy)
bush = text_cleaner(bush)
clinton = text_cleaner(clinton)
obama = text_cleaner(obama)

In [0]:
# Parse the cleaned inaugurals. This can take a bit.
nlp = spacy.load('en')
washington_doc = nlp(washington)
kennedy_doc = nlp(kennedy)
bush_doc = nlp(bush)
clinton_doc = nlp(clinton)
obama_doc = nlp(obama)

In [0]:
# Group into sentences.
washington_sents = [[sent, "Washington"] for sent in washington_doc.sents]
kennedy_sents = [[sent, "Kennedy"] for sent in kennedy_doc.sents]
bush_sents = [[sent, "Bush"] for sent in bush_doc.sents]
clinton_sents = [[sent, "Clinton"] for sent in clinton_doc.sents]
obama_sents = [[sent, "Obama"] for sent in obama_doc.sents]

In [10]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(washington_sents + kennedy_sents + bush_sents + clinton_sents + obama_sents)
sentences.head()

,0,1
0,"(Fellow, citizens, ,, I, am, again, called, up...",Washington
1,"(When, the, occasion, proper, for, it, shall, ...",Washington
2,"(Previous, to, the, execution, of, any, offici...",Washington
3,"(This, oath)",Washington
4,"(I, am, now, about, to, take, ,, and, in, your...",Washington


##Bag of Words

In [0]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
   

In [0]:
# Set up the bags.
washingtonwords = bag_of_words(washington_doc)
kennedywords = bag_of_words(kennedy_doc)
bushwords = bag_of_words(bush_doc)
clintonwords = bag_of_words(clinton_doc)
obamawords = bag_of_words(obama_doc)

# Combine bags to create a set of unique words.
common_words = set(washingtonwords + kennedywords + bushwords + clintonwords + obamawords)

In [13]:
list(common_words)[0]

'president'

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=common_words)
df = pd.DataFrame()
df['text_sentence'] = sentences[0]
df['text_source'] = sentences[1]
#String version of the spacy objects
df['str_sentence'] = [_.text for _ in df.text_sentence]

X = vectorizer.fit_transform(df['str_sentence'])

df2 =  pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

word_counts = pd.concat([df,df2], axis=1)
print(word_counts.shape)
word_counts.head()

(430, 1470)


,text_sentence,text_source,str_sentence,'s,-PRON-,"1,000",100,18th,19th,200,...,wright,writ,write,wrong,year,yearn,yes,yet,yield,young
0,"(Fellow, citizens, ,, I, am, again, called, up...",Washington,"Fellow citizens, I am again called upon by the...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(When, the, occasion, proper, for, it, shall, ...",Washington,"When the occasion proper for it shall arrive, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Previous, to, the, execution, of, any, offici...",Washington,Previous to the execution of any official act ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(This, oath)",Washington,This oath,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(I, am, now, about, to, take, ,, and, in, your...",Washington,"I am now about to take, and in your presence: ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###Supervised Learning model

####Using Random Forest



In [15]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split, cross_val_score

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source','str_sentence'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)


train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
print('\nCross validation score', cross_val_score(rfc,X,Y,cv=3))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9922480620155039

Test set score: 0.37209302325581395

Cross validation score [0.34482759 0.29166667 0.32624113]


Looks like the average is 35%  using Random Forest.

####Using Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('\nCross validation score:', cross_val_score(lr, X, Y, cv=3))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(258, 1467) (258,)
Training set score: 0.9806201550387597

Test set score: 0.4418604651162791

Cross validation score: [0.36551724 0.375      0.32624113]


Average score using Logistic Regression is about 35%

####Using Random Forest with parameter tuning

In [17]:
rfc1 = ensemble.RandomForestClassifier(n_estimators = 500, max_depth = 100)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source','str_sentence'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc1.fit(X_train, y_train)

print('Training set score:', rfc1.score(X_train, y_train))
print('\nTest set score:', rfc1.score(X_test, y_test))
print('\nCross validation score', cross_val_score(rfc1,X,Y,cv=3))

Training set score: 0.9961240310077519

Test set score: 0.4127906976744186

Cross validation score [0.4137931  0.29861111 0.39007092]


Tuning parameters changed average score to 36% for RFC. Let us try Grid search

####Using Grid Search on Random Forest

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}
clf = GridSearchCV(rfc, param_grid, cv=5)
clf.fit(X_train, y_train)
print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
print('\nCross validation score', cross_val_score(clf,X,Y,cv=3))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Training set score: 0.9961240310077519

Test set score: 0.4186046511627907


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from Tru


Cross validation score [0.4        0.29861111 0.36170213]


Average for RFC now went upto 41%, which is a 5% increase.

####Using Grid Search on Logistic Regression

In [19]:
grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
model_lr = GridSearchCV(lr, param_grid=grid_values,cv=5)
model_lr.fit(X_train, y_train)
print('Training set score:', model_lr.score(X_train, y_train))
print('\nTest set score:', model_lr.score(X_test, y_test))
print('\nCross validation score', cross_val_score(model_lr,X,Y,cv=3))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/

Training set score: 0.9961240310077519

Test set score: 0.37209302325581395


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/


Cross validation score [0.35862069 0.35416667 0.29787234]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model

Looks like RFC is better, since cross validation for Logistic regression remains around 35%

## TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than .X of the paragraphs
                             min_df=4, # only use words that appear at least n times
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
str_tfidf=vectorizer.fit_transform(df['str_sentence'])
print("Number of features: %d" % str_tfidf.get_shape()[1])

Number of features: 178


In [24]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer.  We are going to reduce the feature space
svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_lsa = lsa.fit_transform(str_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_lsa,index=df.str_sentence)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Percent variance captured by all components: 83.96006419500355
Component 0:
str_sentence
And if a beachhead of cooperation may push back the jungle of suspicion, let both sides join in creating a new endeavor, not a new balance of power, but a new world of law, where the strong are just and the weak secure and the peace preserved.                                                                                                                  0.535887
May those generations whose faces we cannot yet see, whose names we may never know, say of us here that we led our beloved land into a new century with the American Dream alive for all her children; with the American promise of a more perfect union a reality for all her people; with America's bright flame of freedom spreading throughout all the world.    0.528610
With a new vision of government, a new sense of responsibility, a new spirit of community, we will sustain America's journey.                                                      

####Using Random Forest on TFIDF

In [25]:
rfc = ensemble.RandomForestClassifier()
Y = df.text_source
X = X_lsa

train = rfc.fit(X,Y)
print('\nTest set score:', cross_val_score(rfc,X, Y, cv=3))


Test set score: [0.34482759 0.43055556 0.36879433]


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Scores are similar to Bag of Words for RFC

####Using Random Forest with parameter tuning on TFIDF

In [26]:
#Random Forest attempt
rfc = ensemble.RandomForestClassifier(n_estimators=250,
                                     max_depth=100)
train = rfc.fit(X, Y)
print('\nTest set score:', cross_val_score(rfc,X, Y, cv=3))


Test set score: [0.44137931 0.46527778 0.39007092]


This might be a good consistent score for RFC with parameter tuning. Aveage of around 43%

####Using Logistic Regression on TFIDF

In [27]:
lr = LogisticRegression(solver='liblinear')
train = lr.fit(X, Y)
print('\nCV score:', cross_val_score(lr,X, Y, cv=3))


CV score: [0.40689655 0.39583333 0.36170213]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


This logistic regressor gave a better score compared to bag of words

#Conclusion:

We used two methods here Bag of words and TFIDF. The Bag of words method had average score of 35% - 38% even with parameter tuning and Grid search on RFC, which was optimal model for this method. 
Whereas, TFIDF gave better scores for both Logistic and RFC models. 
Overall I would choose RFC with parameter tuning on TFIDF since scores were consistent and higher than other models.